In [1]:
import os
import cv2
import random


In [2]:
def load_and_preprocess_images(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))  
            images.append(img)
            labels.append(label)
    return images, labels

In [3]:
mask_folder = "/home/muntez/projectwork/CVPR Final Assignment 2/Dataset/Mask"
non_mask_folder = "/home/muntez/projectwork/CVPR Final Assignment 2/Dataset/Non Mask"

mask_images, mask_labels = load_and_preprocess_images(mask_folder, label=1)
non_mask_images, non_mask_labels = load_and_preprocess_images(non_mask_folder, label=0)


all_images = mask_images + non_mask_images
all_labels = mask_labels + non_mask_labels


In [4]:


data = list(zip(all_images, all_labels))
random.shuffle(data)
all_images, all_labels = zip(*data)


split_ratio = 0.7
split_index = int(len(all_images) * split_ratio)

train_images = all_images[:split_index]
train_labels = all_labels[:split_index]
test_images = all_images[split_index:]
test_labels = all_labels[split_index:]


In [5]:
import numpy as np

train_images = np.array(train_images) / 255.0
test_images = np.array(test_images) / 255.0

train_labels = np.array(train_labels)
test_labels = np.array(test_labels)


In [6]:
train_images.shape

(704, 224, 224, 3)

In [7]:
test_images.shape

(302, 224, 224, 3)

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

base_model = tf.keras.applications.MobileNet(input_shape=[224,224,3], weights = "imagenet", include_top=False)

base_model.trainable = False

model = Flatten()(base_model.output)
model = Dense(units=256, activation="relu")(model)
model = Dense(units=64, activation="relu")(model)
prediction_layer = Dense(units=1, activation="sigmoid")(model)

model = Model(inputs = base_model.input, outputs = prediction_layer)
model.compile(optimizer='SGD',loss='binary_crossentropy',metrics=['accuracy'])

2023-08-27 21:40:00.675145: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-27 21:40:01.177035: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-08-27 21:40:01.763252: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-27 21:40:01.780935: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizati  (None, 112, 112, 32)      128       
 on)                                                             
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D  (None, 112, 112, 32)      288       
 )                                                               
                                                                 
 conv_dw_1_bn (BatchNormali  (None, 112, 112, 32)      128   

In [10]:
epochs = 100
batch_size = 32

early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

history = model.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size,
                    validation_split=0.3, callbacks=[early_stopping])


test_loss, test_accuracy = model.evaluate(test_images, test_labels)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Epoch 1/100


2023-08-27 21:40:14.836700: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-27 21:40:15.336000: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f82048b31c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-27 21:40:15.336019: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2023-08-27 21:40:15.388793: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


16/16 [==============================] - 3s 116ms/step - loss: 0.7480 - accuracy: 0.8394 - val_loss: 0.0500 - val_accuracy: 0.9858
Epoch 2/100
16/16 [==============================] - 1s 49ms/step - loss: 0.0144 - accuracy: 0.9959 - val_loss: 0.0427 - val_accuracy: 0.9858
Epoch 3/100
16/16 [==============================] - 1s 45ms/step - loss: 0.0055 - accuracy: 1.0000 - val_loss: 0.0438 - val_accuracy: 0.9858
Epoch 4/100
16/16 [==============================] - 1s 45ms/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.0444 - val_accuracy: 0.9858
Epoch 5/100
10/10 [==============================] - 1s 50ms/step - loss: 0.0435 - accuracy: 0.9901
Test Loss: 0.043517641723155975
Test Accuracy: 0.9900662302970886


In [11]:
model_dir = "/home/muntez/projectwork/CVPR Final Assignment 2/FaceMaskTrain.h5"
tf.keras.models.save_model(model, model_dir)

/tmp/ipykernel_19093/781157402.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, model_dir)
